### SharedMemory

In [63]:
from multiprocessing import shared_memory
shm_a = shared_memory.SharedMemory(create=True, size=10, name = "A")
type(shm_a.buf)

memoryview

In [3]:
buffer = shm_a.buf
len(buffer)

10

In [4]:
# byte 수정
buffer[:4] = bytearray([22, 33, 44, 55])
buffer[4] = 100
buffer

In [75]:
shm_b = shared_memory.SharedMemory(name="A")
buf_b = shm_b.buf

In [62]:
import array
array.array('b', shm_b.buf[:5])
shm_b.buf[:5] = b'howdy'
bytes(shm_b.buf[:5])

FileNotFoundError: [Errno 2] No such file or directory: '/psm_30e969e4'

In [50]:
# 메모리 해제
shm_b.close()
shm_a.close()
# shm_b.unlink() # 없으면 오류남
# shm_a.unlink()

In [15]:
import numpy as np
from multiprocessing.shared_memory import SharedMemory

In [58]:
a = np.array([1,2,3,4,5,6])
shm = SharedMemory(create=True, size=a.nbytes)
b = np.ndarray(a.shape, dtype=a.dtype, buffer=shm.buf)
b[:] = a[:] # shallow copy
b

array([1, 2, 3, 4, 5, 6])

In [18]:
print("b type : ", type(b))
print("a type : ", type(a))

b type :  <class 'numpy.ndarray'>
a type :  <class 'numpy.ndarray'>


In [19]:
shm.name # key를 잉의로 지정함

'psm_c6ab22e3'

Sharememory에 접근해서 데이터 수정

In [61]:
connect_shm = SharedMemory(name=shm.name) # 기존 공유 메모리에 연결
c = np.ndarray((6, ), dtype=np.int64, buffer=connect_shm.buf) # connect_shm.buf 사이즈를 넘으면 오류남
c

<class 'memoryview'>


array([1, 2, 3, 4, 5, 6])

In [30]:
c[-1] = 888
c

array([  1,   2,   3,   4,   5, 888])

In [31]:
b

array([  1,   2,   3,   4,   5, 888])

In [32]:
a

array([1, 2, 3, 4, 5, 6])

In [52]:
# close
connect_shm.close()
shm.close()
# shm.unlink()

### SharedMemoryManager
가변 데이터

In [34]:
from multiprocessing.managers import SharedMemoryManager
smm = SharedMemoryManager()
# 공유 메모리 블럭을 관리하는 프로세스 시작
smm.start()

In [35]:
sl = smm.ShareableList(range(4))
sl

ShareableList([0, 1, 2, 3], name='psm_b7225c67')

In [36]:
raw_shm = smm.SharedMemory(size=128)
another_sl = smm.ShareableList('alpha')
another_sl

ShareableList(['a', 'l', 'p', 'h', 'a'], name='psm_80076d1f')

출처: https://daphne-dev.github.io/2020/09/19/python-multiproccessing/

### Queue

In [37]:
from multiprocessing import Process, Queue
import time

In [54]:
def worker(id, number, q):
    increased_number = 0
    for _ in range(number):
        increased_number += 1
    q.put(increased_number)
    return 

In [55]:
start = time.time()
q = Queue()
processes= []
processes.append(Process(target=worker, args=(1, 50000000, q)))
processes.append(Process(target=worker, args=(2, 50000000, q)))
[p.start() for p in processes]
[p.join() for p in processes]

q.put("EXIT")
total = 0
while True:
    tmp = q.get()
    if tmp == "EXIT":
        break
    else:
        total += tmp
print(time.time()-start)
print(total)

21.592480421066284
1000000000


### Semaphore

In [8]:
from multiprocessing import shared_memory, Semaphore, Process
import time
import numpy  as np

def worker(id, number, shm, arr, sem):
    increased_number = 0
    for _ in range(number):
        increased_number += 1
    sem.acquire() # 세마포어 획득
    new_shm = shared_memory.SharedMemory(name=shm) # 공유 메모리 가져옴
    tmp_arr = np.ndarray(arr.shape, dtype=arr.dtype, buffer=new_shm.buf)
    print(tmp_arr)
    tmp_arr[0] += increased_number
    sem.release()

In [9]:
start = time.time()

arr = np.array([0])
shm = shared_memory.SharedMemory(create=True, size=arr.nbytes)
np_shm = np.ndarray(arr.shape, dtype=arr.dtype, buffer=shm.buf)
sem = Semaphore()

processes= []
processes.append(Process(target=worker, args=(1, 50000000, shm.name, np_shm, sem)))
processes.append(Process(target=worker, args=(2, 50000000, shm.name, np_shm, sem)))
processes.append(Process(target=worker, args=(3, 50000000, shm.name, np_shm, sem)))

[p.start() for p in processes]
[p.join() for p in processes]

print(time.time()-start)
print(np_shm, np_shm[0])
shm.close() # 공유 메모리 사용 종료
shm.unlink() # 공유 메모리 블럭 삭제

[0]
[50000000]
[100000000]
2.3690385818481445
[150000000] 150000000


In [24]:
from multiprocessing import shared_memory, Semaphore, Process
import time
import numpy  as np

def put(name, data):
    conn_sm = shared_memory.SharedMemory(name=name)
    new_arr = np.ndarray(data.shape, dtype=data.dtype, buffer=conn_sm.buf)
    new_arr[:] = data[:] # copy
    print(new_arr)

def test(id, shm, arr, sem):

    new_shm = shared_memory.SharedMemory(name=shm) # 공유 메모리 가져옴
    tmp_arr = np.ndarray(arr.shape, dtype=arr.dtype, buffer=new_shm.buf)
    print(f"{id}: Data >>>>>>>>>>>>>>>>>>>> ", tmp_arr)


In [30]:
arr = np.array([1,2,3,4,5])
sm = shared_memory.SharedMemory(create=True, size=arr.nbytes, name="sample")
put("sample", np.array([6,5,4,3,2]))

sem = Semaphore()

processes= []
processes.append(Process(target=test, args=(1, "sample", np_arr, sem)))
processes.append(Process(target=test, args=(2, "sample", np_arr, sem)))
processes.append(Process(target=test, args=(3, "sample", np_arr, sem)))

[p.start() for p in processes]
[p.join() for p in processes]


[6 5 4 3 2]
1: Data >>>>>>>>>>>>>>>>>>>>  2: Data >>>>>>>>>>>>>>>>>>>> [6 5 4 3 2] 
[6 5 4 3 2]
3: Data >>>>>>>>>>>>>>>>>>>>  [6 5 4 3 2]


[None, None, None]

In [36]:
sm.close()
sm.unlink()

In [105]:
# create SM and copy array
arr = np.array([1,2,3,4,5])
sm = shared_memory.SharedMemory(create=True, size=arr.nbytes, name="sample")
np.ndarray(arr.shape, dtype=arr.dtype, buffer=sm.buf)[:] = arr[:]
shape, dtype = arr.shape, arr.dtype

In [102]:
# connect to shared memory
conn_sm = shared_memory.SharedMemory(name="sample")
print(np.ndarray(shape, dtype=dtype, buffer=conn_sm.buf))

[1 2 3 4 5]


In [96]:
sm.close()
sm.unlink()

In [109]:
check = shared_memory.SharedMemory(name="sample")
check.close()
# check.unlink()
try:
    shared_memory.SharedMemory(name="sample")
    print("sample stil alive")
except:
    print("sample removed successfully")

FileNotFoundError: [Errno 2] No such file or directory: '/sample'